## Import

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

## Load train.csv

In [2]:
df = pd.read_csv('train.csv')
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


## Remove duplicats

In [3]:
def reset_step_for_duplicated_sessions(df):
    """ Reset the step for some bugged session in which the step restart from 1 in some random interaction """
    res_df = df.copy()
    # find the sessions in which the step restarts at some point
    df_dup = df[["session_id", "user_id", "step"]]
    df_dup = df_dup[df_dup["step"] == 1]
    df_dup = df_dup.groupby(['user_id','session_id','step']).size() \
        .sort_values(ascending=False) \
        .reset_index(name='count')
    df_dup = df_dup[df_dup["count"] > 1]
    df_dup = df_dup[['user_id','session_id']]

    # reset the steps for the duplicated-steps sessions
    for _,row in tqdm(df_dup.iterrows()):
        mask = (df.user_id == row.user_id) & (df.session_id == row.session_id)
        sess_length = sum(mask *1)
        res_df.loc[mask, 'step'] = np.arange(1, sess_length+1, dtype='int')

    return res_df

df_without_duplicats = reset_step_for_duplicated_sessions(df)

### Check duplicats

In [4]:
df_without_duplicats[df.session_id == '15e8515f6d309']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
1686470,C015B3HYIZSF,15e8515f6d309,1541263488,1,clickout item,1008733,BR,"Gramado, Brazil",mobile,NaN,1008733|655716|4342348|502066|1352530|507861|4...,136|115|93|135|92|71|112|108|101|92|127|122|15...
2177386,C015B3HYIZSF,15e8515f6d309,1541345795,2,interaction item image,6452918,BR,"Gramado, Brazil",mobile,NaN,NaN,NaN
2177387,C015B3HYIZSF,15e8515f6d309,1541345795,3,interaction item image,6452918,BR,"Gramado, Brazil",mobile,NaN,NaN,NaN
2177388,C015B3HYIZSF,15e8515f6d309,1541345797,4,clickout item,6452918,BR,"Gramado, Brazil",mobile,NaN,655716|507861|3133206|2258514|1673835|3583050|...,283|156|133|208|196|246|208|222|199|59|160|128...
2177389,C015B3HYIZSF,15e8515f6d309,1541345934,5,clickout item,655716,BR,"Gramado, Brazil",mobile,NaN,655716|507861|3133206|6147882|1673835|2258514|...,127|65|57|49|85|70|58|179|101|120|56|99|89|23|...
2177390,C015B3HYIZSF,15e8515f6d309,1541360791,6,clickout item,655716,BR,"Gramado, Brazil",mobile,NaN,502066|655716|4342348|1352530|3403842|2143854|...,62|69|83|78|61|90|76|65|69|52|113|56|82|101|40...
2177391,C015B3HYIZSF,15e8515f6d309,1541361109,7,clickout item,3403842,BR,"Gramado, Brazil",mobile,NaN,502066|655716|4342348|1352530|3403842|2143854|...,62|69|83|78|61|90|76|65|69|52|113|56|82|101|40...
2177392,C015B3HYIZSF,15e8515f6d309,1541361389,8,clickout item,6147882,BR,"Gramado, Brazil",mobile,NaN,502066|655716|4342348|1352530|3403842|2143854|...,62|69|83|78|61|90|76|65|69|52|113|56|82|101|40...
2177393,C015B3HYIZSF,15e8515f6d309,1541361591,9,clickout item,1405084,BR,"Gramado, Brazil",mobile,NaN,502066|655716|4342348|1352530|3403842|2143854|...,62|69|83|78|61|90|76|65|69|52|113|56|82|101|40...


## Get number of sessions

In [5]:
session_ids = df_without_duplicats.session_id.unique()
num_of_sessions = len(session_ids)
num_of_sessions

910683

## Get index for spliting data

In [6]:
stamp = 0.8
train_sessions_number = int(stamp * num_of_sessions)
train_sessions_number

728546

In [7]:
train_end_session_id = session_ids[train_sessions_number]
train_end_session_id

'cae8906db6f21'

### Get last train session rows

In [8]:
df_without_duplicats[df_without_duplicats.session_id == train_end_session_id]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
12756474,XI4V09UQBBN2,cae8906db6f21,1541491756,1,search for item,2207100,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756475,XI4V09UQBBN2,cae8906db6f21,1541491836,2,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756476,XI4V09UQBBN2,cae8906db6f21,1541491836,3,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756477,XI4V09UQBBN2,cae8906db6f21,1541491846,4,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756478,XI4V09UQBBN2,cae8906db6f21,1541491846,5,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756479,XI4V09UQBBN2,cae8906db6f21,1541491846,6,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756480,XI4V09UQBBN2,cae8906db6f21,1541491846,7,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756481,XI4V09UQBBN2,cae8906db6f21,1541491846,8,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756482,XI4V09UQBBN2,cae8906db6f21,1541491846,9,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN
12756483,XI4V09UQBBN2,cae8906db6f21,1541491846,10,interaction item image,5740296,TW,"Renai Township, Taiwan",desktop,NaN,NaN,NaN


### Get last index for training set

In [9]:
train_last_index = int(df_without_duplicats[df_without_duplicats.session_id == train_end_session_id][-1:].index[0])
train_last_index

12756502

## Split data to train and test

### Create train data frame

In [10]:
df_train = df_without_duplicats[:train_last_index + 1]
df_train.tail()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
12756498,XI4V09UQBBN2,cae8906db6f21,1541491922,25,interaction item image,5740296,TW,"Nantou City, Taiwan",desktop,NaN,NaN,NaN
12756499,XI4V09UQBBN2,cae8906db6f21,1541491925,26,change of sort order,price only,TW,"Nantou City, Taiwan",desktop,NaN,NaN,NaN
12756500,XI4V09UQBBN2,cae8906db6f21,1541491925,27,interaction item image,5740296,TW,"Nantou City, Taiwan",desktop,NaN,NaN,NaN
12756501,XI4V09UQBBN2,cae8906db6f21,1541491925,28,filter selection,Sort by Price,TW,"Nantou City, Taiwan",desktop,Sort by Price,NaN,NaN
12756502,XI4V09UQBBN2,cae8906db6f21,1541491953,29,clickout item,3857806,TW,"Nantou City, Taiwan",desktop,Sort by Price,4182178|3861994|5832464|1908423|5740296|671947...,64|65|74|80|82|84|85|85|85|86|87|87|88|91|91|9...


### Create test data frame

In [11]:
df_test = df_without_duplicats[train_last_index + 1:]
df_test.tail()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
15932987,ZYNMLE3MV3LK,62728015bec05,1541544490,15,interaction item image,6617798,PT,"Paris, France",desktop,NaN,NaN,NaN
15932988,ZYNMLE3MV3LK,62728015bec05,1541544491,16,clickout item,6617798,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...
15932989,ZYNMLE3MV3LK,62728015bec05,1541544540,17,clickout item,2712342,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...
15932990,ZYNMLE3MV3LK,62728015bec05,1541544967,18,change of sort order,interaction sort button,PT,"Paris, France",desktop,NaN,NaN,NaN
15932991,ZYNMLE3MV3LK,62728015bec05,1541544973,19,clickout item,1161323,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...


### Create ground truth

In [12]:
ground_truth = df_without_duplicats[train_last_index + 1:]
df_test.tail(1)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
15932991,ZYNMLE3MV3LK,62728015bec05,1541544973,19,clickout item,1161323,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...


## Set NaN to last clickout foor user - df_test

### Get all session ids from test df

In [13]:
grouped_session = df_test.groupby(['user_id', 'session_id']).agg(
    max_timestamp = ('timestamp', max)
)
grouped_session.reset_index(level=0, inplace=True)

### Get sessions to change last clickout

In [14]:
session_ids_to_change = []
user_ids = grouped_session.user_id.unique()
for user_id in tqdm(user_ids):
    last_session_id = str(grouped_session[grouped_session.user_id == user_id]
                          .sort_values('max_timestamp').head(1).index[0])
    session_ids_to_change.append(last_session_id)
session_ids_to_change
    

['d4c5cc7f469d7',
 'ddabdf53a4d38',
 'f9eedbf105c5c',
 'b0c0d9a424374',
 '72b8fed0dfbaa',
 '73ad1e08b4430',
 'a776605c278b0',
 'bd9c40e12933c',
 '42ae8c6fa3dbe',
 'fdee89d524de6',
 'd6f5e96b7cc7a',
 'e6c7b554aecc0',
 '54f8a2c8da7aa',
 '4227b4eb56fbb',
 'eb2b372a622f8',
 '7e5ab77bb58f3',
 'bf44be699e090',
 '567364f571511',
 '54c56648ae40a',
 'ca41b44cc4c71',
 'c1d980f00e68b',
 '1aa18f4d38589',
 'af3b8feb81a98',
 '75682c9a2986c',
 'a52f015ae162b',
 '4bbd2652d9bea',
 '907f5a354f5bd',
 'f4035569d50cd',
 '402fd7dfec659',
 '5736b4003aa39',
 'd94de76870e37',
 '019bc1876ef86',
 'd204cdc464ab7',
 '9e9fe69262ec6',
 'fb64c101d0539',
 '35fbc5f86654f',
 'd6ea7d08a39b6',
 '98e6907df781f',
 'cfd92344175ef',
 '74364758477cb',
 '180ba5497a0b3',
 '6a94c2dd97cbb',
 '2b692ba1c3787',
 'c0cc41fdc0b7c',
 '4fe3cd4d60d16',
 'a02c2031bad92',
 '8425dea64cd76',
 'a32f1912504f5',
 '1fc401eab71e9',
 '44ae89845c7ec',
 '154e87badb386',
 'a7205128ab534',
 'a7fd43ade6fd0',
 'ca28af151382d',
 'fb2dcc343518d',
 'dcd8f558

### Set last clickout reference in user's last session to NaN

In [15]:
df_test.loc[(df_test.session_id.isin(session_ids_to_change)) & (df_test.action_type == 'clickout item'), 'reference'] = np.nan


/home/mscib/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Test replacing 

In [16]:
df_test[df_test.session_id == 'ff27adc23f6cb']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13274122,057C9BJOBPK3,ff27adc23f6cb,1541284125,1,search for destination,"Santa Ponsa, Spain",UK,"Santa Ponsa, Spain",desktop,NaN,NaN,NaN
13274123,057C9BJOBPK3,ff27adc23f6cb,1541284178,2,interaction item image,127534,UK,"Santa Ponsa, Spain",desktop,NaN,NaN,NaN
13274124,057C9BJOBPK3,ff27adc23f6cb,1541284178,3,interaction item image,127534,UK,"Santa Ponsa, Spain",desktop,NaN,NaN,NaN
13274125,057C9BJOBPK3,ff27adc23f6cb,1541284203,4,interaction item deals,127534,UK,"Santa Ponsa, Spain",desktop,NaN,NaN,NaN
13274126,057C9BJOBPK3,ff27adc23f6cb,1541284225,5,clickout item,NaN,UK,"Santa Ponsa, Spain",desktop,NaN,128111|37323|127534|37328|7447|37321|127372|74...,113|86|53|159|170|107|102|73|98|94|46|48|87|56...
13274127,057C9BJOBPK3,ff27adc23f6cb,1541286128,6,interaction item deals,37324,UK,"Santa Ponsa, Spain",desktop,NaN,NaN,NaN
13274128,057C9BJOBPK3,ff27adc23f6cb,1541286139,7,interaction item deals,37324,UK,"Santa Ponsa, Spain",desktop,NaN,NaN,NaN


## Set NaN to 0

In [17]:
df_test = df_test.fillna(0)
df_train = df_train.fillna(0)
ground_truth = ground_truth.fillna(0)

## Save test data set 

In [18]:
df_test.to_csv('my_test.csv')
df_train.to_csv('my_train.csv')
ground_truth.to_csv('groundTruth.csv')